In [312]:
import redis
import json 
import numpy as np
import pandas as pd
import pprint
from collections import Counter
#使用redis

In [313]:
r = redis.StrictRedis(host='redis', port=6379, db=0)
#使用連線 ip='redis' port=6379 db=0

In [314]:
with open('./jsondata/bigtable_1.2.json') as f:
    bigTable=json.load(f)
with open('./jsondata/final_tainan.json') as t:
    tainanData=json.load(t)
with open('./jsondata/TainanGoodDien_1.0.json') as t:
    goodDien=json.load(t)
#匯入各種資料
with open('./dict/MyDict.csv',encoding='big5') as t:
    mydict=t.read()
#自己的字典for同義字

In [315]:
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]
pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]

pinsynonymsdict['飲料']='飲料專賣店'
pinsynonymsdict['喝的']='飲料專賣店'
pinsynonymsdict['涼的']='飲料專賣店'

pinsynonymsdict['綜子']='肉粽'

pinsynonymsdict['冰品']='剉冰、豆花'
pinsynonymsdict['冰']='剉冰、豆花'
pinsynonymsdict['豆花']='剉冰、豆花'
pinsynonymsdict['剉冰']='剉冰、豆花'

pinsynonymsdict['日式']='綜合日式料理'

# pinsynonymsdict['生魚片']='生魚片、壽司專賣'
# pinsynonymsdict['壽司']='生魚片、壽司專賣'

pinsynonymsdict['燒烤']='炭烤串燒'
pinsynonymsdict['串燒']='炭烤串燒'
pinsynonymsdict['串烤']='炭烤串燒'

pinsynonymsdict['下午茶']='西點烘焙、麵包'
pinsynonymsdict['點心']='西點烘焙、麵包'
pinsynonymsdict['甜點']='西點烘焙、麵包'

pinsynonymsdict
#取得同義字字典

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '下午茶': '西點烘焙、麵包',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美

In [316]:
for i in r.keys():
    r.delete(i)
for word in pinsynonymsdict:
    if pinsynonymsdict[word]=='否食物美味':
        pinsynonymsdict[word]='食物不好吃'
    elif pinsynonymsdict[word]=='否划算':
        pinsynonymsdict[word]='價格較高昂'
    elif pinsynonymsdict[word]=='否環境好':
        pinsynonymsdict[word]='環境比較不好'
    elif pinsynonymsdict[word]=='否服務好':
        pinsynonymsdict[word]='服務較差'
    elif pinsynonymsdict[word]=='否其他正面情緒':
        pinsynonymsdict[word]='評論有些負面情緒詞'
#將否情緒改成tag用詞


pinsynonymsdict

{'CP值很低': '價格較高昂',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '價格較高昂',
 'cp值太低': '價格較高昂',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '服務較差',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '食物不好吃',
 '下午茶': '西點烘焙、麵包',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '環境比較不好',
 '不划算': '價格較高昂',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '食物不好吃',
 '不太優': '食物不好吃',
 '不太喜歡': '食物不好吃',
 '不太好吃': '食物不好吃',
 '不太愛吃': '食物不好吃',
 '不太慣': '食物不好吃',
 '不好吃': '食物不好吃',
 '不對味': '食物不好吃',
 '不怎麼樣': '食物不好吃',
 '不想離開': '其他正面情緒',
 '不愛': '食物不好吃',
 '不愛吃': '食物不好吃',
 '不愛它': '食物不好吃',
 '不搭': '食物不好吃',
 '不敢恭維': '環境比較不好',
 '不斐': '價格較高昂',
 '不新鮮': '食物不好吃',
 '不會再來': '評論有些負面情緒詞',
 '不會去第二次': '評論有些負面情緒詞',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '食物不好吃',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '價格較高昂',
 '不脆': '

In [317]:
dienscore={dien['name']:round(dien['score'],2) for dien in bigTable}
dienscore
#kv >店:分數 排序用

{'同記安平豆花(安平總店)': 6.03,
 '深藍咖啡館': 8.32,
 '阿堂鹹粥': 8.13,
 '富盛號碗粿': 10.85,
 '依蕾特布丁奶酪(安平門市)': 8.95,
 '食下有約 想法廚房': 34.61,
 '飛饗創意西式料理': 35.99,
 '莉莉水果店': 8.73,
 '連得堂餅舖': 3.51,
 '矮仔成蝦仁飯': 9.43,
 '泰成水果店': 11.11,
 '騷烤家(前鋒總店)': 27.63,
 '阿松割包': 11.62,
 '秉醇烘焙坊': 18.17,
 '冰鄉': 7.53,
 '小銅鍋義式餐廳(德安誠品店)': 42.08,
 '周氏蝦捲(總店)': 10.43,
 '大菜市純薏仁': 9.68,
 '聯盈發點心專門店': 14.9,
 'OMAYA春川炒雞(台南永華店)': 28.46,
 '西井村蜂蜜滷味(安平店)': 17.22,
 '瑞益行': 16.92,
 '六千牛肉湯': 11.72,
 '文章牛肉湯': 10.33,
 '拿滋派瑞 Nuts Pirates': 17.75,
 '陳家蚵捲': 9.62,
 'EVA伊娃日式泡芙專賣店': 8.92,
 'A WEEK PANCAKE&COFFEE': 23.33,
 '邱家小卷米粉': 7.79,
 '阿裕現宰牛肉涮涮鍋': 13.62,
 '山根壽司': 13.76,
 '金得春捲': 11.48,
 '克林台包': 5.14,
 'Doga香酥脆椒': 6.59,
 '懷舊小棧': 7.05,
 'FUN Tower日式可麗餅': 20.93,
 '阿憨鹹粥': 8.32,
 '花見宇治抹茶甘味処': 18.36,
 '鹿早茶屋': 16.1,
 '窩吧': 19.67,
 'Kadoya喫茶店': 14.3,
 '王西勢食品行': 7.58,
 '窄門咖啡': 2.39,
 '秀治咖啡簡餐(臨安店)': 20.67,
 '橋北屋日本家庭料理(永福店)': 12.97,
 '歐加里(台南大學店)': 38.98,
 '華侖婷娜': 13.13,
 '阿明豬心': 12.11,
 '旗哥牛肉湯': 7.68,
 '裕成水果店': 4.78,
 '太陽牌冰品': 10.66,
 '赤崁棺材板': 12.36,
 '保哥黑輪': 6.18,
 '

In [318]:

dientag={dien['name']:set(dien['tags']) for dien in bigTable}
#kv >店:tagset
dientag

{'同記安平豆花(安平總店)': {'剉冰、豆花', '同記安平豆花', '安平', '服務較差', '珍珠', '紅豆', '豆花'},
 '深藍咖啡館': {'千層', '栗子', '派', '深藍', '蛋糕', '西點烘焙、麵包', '非常貴', '食物美味'},
 '阿堂鹹粥': {'價格較高昂', '其他小吃', '粥', '虱目', '阿堂', '食物美味', '魚腸', '鹹粥'},
 '富盛號碗粿': {'其他小吃', '安平', '富盛號', '小份量', '民族', '碗粿', '食物美味', '魚羹'},
 '依蕾特布丁奶酪(安平門市)': {'依蕾特', '典藏', '奶酪', '布丁', '抹茶', '西點烘焙、麵包', '食物美味'},
 '食下有約 想法廚房': {'其他正面情緒',
  '其他異國料理',
  '區郡',
  '想法廚房',
  '環境好',
  '花雕',
  '蹄膀',
  '非常好吃',
  '食下有約',
  '食物很香',
  '食物精緻漂亮'},
 '飛饗創意西式料理': {'其他正面情緒',
  '典藏',
  '土壤',
  '服務好',
  '沙拉',
  '燉',
  '環境超棒',
  '義式料理',
  '非常好吃',
  '食物精緻漂亮',
  '饗'},
 '莉莉水果店': {'價格較高昂', '剉冰、豆花', '前路', '水果', '肉圓', '莉莉', '豆花', '食物美味'},
 '連得堂餅舖': {'休閒零食', '價格較高昂', '兩包', '崇安', '煎餅', '連得堂', '餅家'},
 '矮仔成蝦仁飯': {'價格較高昂', '其他小吃', '矮仔成', '蛋湯', '蝦仁', '蝦仁飯', '食物美味', '鴨蛋'},
 '泰成水果店': {'價格較高昂', '哈密瓜', '哈蜜瓜', '果汁', '正興', '瓜瓜', '芒果', '食物美味'},
 '騷烤家(前鋒總店)': {'其他正面情緒', '府城騷烤家', '油條', '炭烤串燒', '燒烤', '甜不辣', '非常好吃', '騷烤'},
 '阿松割包': {'價格較高昂', '割包', '小份量', '瘦肉', '花生醬', '豬舌', '豬舌包', '食物美味', '麵食點心'},
 '秉醇烘焙坊':

In [319]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
for dien in dientag:
    for pin in pipin:
        if pin[2] in dientag[dien]:
            dientag[dien].add(pin[0])
        if pin[3] in dientag[dien]:
            dientag[dien].add(pin[1])
#將包含的情緒意義加入querytag
dientag

{'同記安平豆花(安平總店)': {'剉冰、豆花', '同記安平豆花', '安平', '服務較差', '珍珠', '紅豆', '豆花'},
 '深藍咖啡館': {'價格較高昂', '千層', '栗子', '派', '深藍', '蛋糕', '西點烘焙、麵包', '非常貴', '食物美味'},
 '阿堂鹹粥': {'價格較高昂', '其他小吃', '粥', '虱目', '阿堂', '食物美味', '魚腸', '鹹粥'},
 '富盛號碗粿': {'其他小吃', '安平', '富盛號', '小份量', '民族', '碗粿', '食物美味', '魚羹'},
 '依蕾特布丁奶酪(安平門市)': {'依蕾特', '典藏', '奶酪', '布丁', '抹茶', '西點烘焙、麵包', '食物美味'},
 '食下有約 想法廚房': {'其他正面情緒',
  '其他異國料理',
  '區郡',
  '想法廚房',
  '環境好',
  '花雕',
  '蹄膀',
  '非常好吃',
  '食下有約',
  '食物很香',
  '食物精緻漂亮',
  '食物美味'},
 '飛饗創意西式料理': {'其他正面情緒',
  '典藏',
  '土壤',
  '服務好',
  '沙拉',
  '燉',
  '環境好',
  '環境超棒',
  '義式料理',
  '非常好吃',
  '食物精緻漂亮',
  '食物美味',
  '饗'},
 '莉莉水果店': {'價格較高昂', '剉冰、豆花', '前路', '水果', '肉圓', '莉莉', '豆花', '食物美味'},
 '連得堂餅舖': {'休閒零食', '價格較高昂', '兩包', '崇安', '煎餅', '連得堂', '餅家'},
 '矮仔成蝦仁飯': {'價格較高昂', '其他小吃', '矮仔成', '蛋湯', '蝦仁', '蝦仁飯', '食物美味', '鴨蛋'},
 '泰成水果店': {'價格較高昂', '哈密瓜', '哈蜜瓜', '果汁', '正興', '瓜瓜', '芒果', '食物美味'},
 '騷烤家(前鋒總店)': {'其他正面情緒',
  '府城騷烤家',
  '油條',
  '炭烤串燒',
  '燒烤',
  '甜不辣',
  '非常好吃',
  '食物美味',
  '騷烤'},
 '阿松割包': {'價格較高昂', '割包

In [320]:
alltag=set()
for dien in dientag:
    alltag.update(dientag[dien])
alltag.remove("")
alltag
#找出所有tag

{'食莊',
 '夢',
 '考駕照',
 '巧味鹹酥雞',
 '筱倩家',
 '桃山',
 '里',
 '差個',
 '檸蜜',
 '浙江',
 '辛魂',
 '租房子',
 '台東',
 '大蛋餅',
 '曼玉',
 '鯨吞',
 '東北',
 '三段',
 '安妮',
 '鮮蔥',
 '江蕙',
 '參酌',
 '肉燉',
 '認滴',
 '阿母',
 '古文字',
 '將太',
 '吠陀',
 '加菜',
 '御食',
 '僑出',
 '阿辰',
 '三鮮',
 '十七層',
 '沒吃會',
 '灰熊',
 '即成',
 '回北',
 '師父',
 '田路',
 '華興',
 '杓',
 '咖滋',
 '剛從',
 '阿彌',
 '毛森江',
 '森野莓',
 '過若',
 '點多到',
 '食物不好吃',
 '一碗水',
 '文具店',
 '資訊',
 '老菜',
 '意臥江',
 '夏花',
 '油量',
 '梅蘭',
 '風壹',
 '湯屋',
 '蝎蠣',
 '友誼',
 '剉冰、豆花',
 '內湖',
 '恆春',
 '麻古茶坊',
 '灰鏡',
 '齊齊',
 '糖廠',
 '第四層',
 '奶餡',
 '廟',
 '雙餡',
 '関',
 '鮮芋',
 '分子',
 '腐皮',
 '凸餅',
 '太笨',
 '心蛋',
 '打翻',
 '蝶豆',
 '水煎包、生煎包',
 '薏仁',
 '湯很先',
 '臉妹',
 '新生北路',
 '能用',
 '全虹',
 '山泉',
 '價格戰',
 '竹園',
 '雙鮮',
 '塗鴉',
 '北港人',
 '吸裡面',
 '加值',
 '塔香豚',
 '中炎山',
 '李太太',
 '堂',
 '棧板',
 '鳩',
 '嗲配',
 '他漲',
 '拉手',
 '塔',
 '巴莎',
 '櫛瓜',
 '法利',
 '彩珠',
 '口罩',
 '直火',
 '過呀',
 '港明',
 '團子',
 '帕尼尼',
 '有間',
 '樓閣',
 '水調歌頭',
 '小夜食堂',
 '瑞吉兒',
 '顆心',
 '客宅配',
 '魚乾',
 '旁巷',
 '湖鹽',
 '自助式',
 '衝上來',
 '高級',
 '雞皮',
 '鄭小姐',
 '的國',
 '猛男',
 '阿姐給',
 '晨曦',
 '酸奶',

In [321]:
table_df=pd.DataFrame(bigTable)

In [322]:
pprint.pprint(dientag)

{'0.618': {'煎餃', '食物美味', '熊貓', '刈包', '蒸蛋', '擂茶', '複合式'},
 '0628全日早午餐店': {'份量大',
                '偏酸',
                '全日',
                '其他正面情緒',
                '司康',
                '小鐵',
                '烘蛋',
                '西式早餐',
                '貓咪',
                '食物精緻漂亮',
                '食物美味'},
 '1004 Brunch 早午餐': {'其他正面情緒',
                     '勾咕',
                     '天使',
                     '帕里尼',
                     '早午餐',
                     '服務好',
                     '環境好',
                     '複合式',
                     '韓國',
                     '食物美味'},
 '1010湘(台南三越西門店)': {'偏辣',
                    '價格較高昂',
                    '服務好',
                    '湖南菜',
                    '湘',
                    '湘菜',
                    '湘菜(湖南菜)',
                    '皮蛋',
                    '西米露',
                    '重口味',
                    '非常好吃',
                    '食物美味'},
 '108巷咖啡': {'迪克', '藍赫法', '查爾斯', '價格較高昂', '早午餐', '芬堡', '班尼'},
 '123冰城': {'四段',

                          '開心果',
                          '食物很香',
                          '食物美味'},
 'PIZZA BAR': {'披薩', '划算', '義式料理', '雙倍', '螺旋', '烤雞肉', '先生'},
 'POP PIE': {'價格較高昂',
             '其他正面情緒',
             '小高',
             '服務好',
             '派',
             '環境好',
             '籽醬',
             '老屋',
             '西點烘焙、麵包',
             '食物很香',
             '食物精緻漂亮',
             '食物美味',
             '鹹'},
 'PS cafe brunch': {'翹', '咖啡專賣', '瑀', '早午餐', '鬆餅', '臀'},
 'PS oven': {'小餐', '西點烘焙、麵包', '小星星', '蛋塔', '早午餐', '瑪德蓮'},
 'Pa Pa Yummy 柴燒美食基地': {'其他正面情緒',
                        '披薩',
                        '新化',
                        '桂圓',
                        '石窯',
                        '窯',
                        '義式料理',
                        '食物很香',
                        '食物美味'},
 'Pain Perdu 邦貝樂啾咖啡館': {'豬排', '瓦城', '食物美味', '泰國', '吉米', '價格較高昂', '勝博殿', '複合式'},
 "Perfetto Caff'e完美咖啡": {'小妹', '完美', '三明治', '其他正面情緒', '咖啡', '鬆餅', '複合式'},
 'Perfume Dance 跳舞香水

                 '灣裡',
                 '環境好',
                 '羊泉',
                 '羊燥',
                 '羊肉',
                 '羊肉湯',
                 '非常好吃',
                 '食物很香',
                 '食物精緻漂亮',
                 '食物美味'},
 '咩 灣裡羊肉(總店)': {'其他正面情緒',
                '咩',
                '服務好',
                '灣裡',
                '熱炒、快炒',
                '環境好',
                '羊泉',
                '羊肉',
                '羊肉爐',
                '非常好吃',
                '食物美味'},
 '咪咪笑串燒': {'穀倉', '炭烤串燒', '飯糰', '優格', '果汁', '茭白'},
 '咿吉麵包坊  隱藏版平價美味麵包店': {'咿吉', '包坊', '菠蘿', '西點烘焙、麵包', '麵', '食物美味', '大和路'},
 '品元扁食水餃': {'肉絲', '五十', '食物很香', '酸豆', '麵食點心', '扁食', '雪菜'},
 '品味羊肉店': {'羊肚', '點大骨湯', '當歸', '吸裡面', '食物不好吃', '羊肉', '食補藥膳餐廳'},
 '品廚牛排(新營店)': {'阿靜', '執迷', '品廚', '引述', '美式料理', '牛排'},
 '品樂菓全日早午餐': {'厚片', '任選', '食物美味', '菓', '品樂', '西式早餐', '布丁'},
 '品田牧場(台南勝利店)': {'偏甜',
                 '其他正面情緒',
                 '台塑',
                 '品田',
                 '日式豬排專賣',
                 '

          '葉子',
          '蝦仁飯',
          '食物美味',
          '麵食點心'},
 '新化牛の屋': {'無評', '新化', '必吃', '晚來一步', '牛肉湯', '好爽', '中式早餐'},
 '新化老攤八寶愛玉冰': {'剉冰、豆花', '新化', '帽子', '紅豆餅', '雞蛋糕', '警察局'},
 '新化老攤雞蛋糕': {'新化', '星光', '紅豆餅', '雞蛋糕', '其他小吃', '價格較高昂', '香腸'},
 '新化老爹牛肉麵': {'', '麵食點心', '無評'},
 '新化葉麥克炸雞': {'太子', '漢堡、炸雞', '新化', '麥克', '食物美味', '鹹酥雞', '炸雞'},
 '新化農舍餐廳': {'新化', '光頭', '農舍', '竹笙湯', '台菜餐廳', '農舍庭園餐廳'},
 '新化阿發排骨大王(善化分店)': {'大王', '紅燒', '其它中式料理', '配菜', '善化', '發排'},
 '新味香小吃': {'', '無評', '其它中式料理'},
 '新園A 汕頭火鍋': {'其他鍋類', '新園', '另一家', '小卷', '美牛', '價格較高昂', '東石'},
 '新天地': {'買五入', '可愛', '人形', '共五入', '泰式料理', '卡士'},
 '新市土地公臭豆腐': {'份量大',
              '價格較高昂',
              '其他小吃',
              '其他正面情緒',
              '土地公',
              '新市',
              '泡菜',
              '臭豆腐',
              '豪記',
              '食物美味'},
 '新市牛肉清湯': {'熱炒、快炒', '牛肉', '新市', '清湯', '牛肉湯', '第一碗'},
 '新市豬腳': {'滷菜', '滷汁', '食物美味', '木棉花', '爌', '其他小吃', '豬腳', '價格較高昂'},
 '新市農會超市': {'新市區', '毛豆', '農會', '冰淇淋', '霜淇淋', '其他正面情緒', '勛'}

 '石原日式料理': {'', '綜合日式料理', '無評'},
 '石堂-極和牛石頭燒': {'', '韓式燒肉', '無評'},
 '石堂極和牛石頭燒': {'極和牛', '環境比較不好', '韓式', '食物美味', '石堂', '價格較高昂', '蛋圈', '韓式燒肉', '烤肉'},
 '石家㊣阿美': {'剉冰、豆花', '綠豆湯', '粉角', '中西區', '阿美', '銅板'},
 '石榴園複合餐飲': {'石榴園', '馬來', '小雞', '豬腳', '咖哩', '複合式'},
 '石橋子羅宋可頌專賣': {'石橋', '芯', '環境比較不好', '冰火', '菠蘿', '西點烘焙、麵包', '可頌'},
 '石精臼牛肉湯': {'牛肉', '小份量', '石精', '臼', '其它中式料理', '食物美味', '牛肉湯', '價格較高昂', '民族'},
 '石精臼米糕': {'小南米糕', '米糕', '小份量', '石精', '樂天', '食物美味', '其他小吃', '小高'},
 '石精臼虎頭蝦捲、排骨酥': {'棺材板', '小份量', '石精', '八神', '其它中式料理', '食物美味', '蝦捲', '小高'},
 '石精臼蚵仔煎': {'小份量', '蚵', '香菇', '食物美味', '其它中式料理', '蚵仔煎', '飯湯', '價格較高昂', '民族'},
 '石舂臼海鮮粥': {'姊姊', '湯尼', '肉粽', '其他小吃', '粽', '舂'},
 '石頭日式炭火燒肉(新營館)': {'炭烤串燒', '彩帶', '乳香', '拔拔', '竹筒飯', '小燕子'},
 '石頭日式炭火燒肉（永康-尊貴館）': {'服務生', '炭烤串燒', '身高', '永華', '食物美味', '石頭', '價格較高昂', '小燕子'},
 '石頭鄉烤玉米': {'燜', '食物美味', '玉米', '石頭', '其他小吃', '價格較高昂', '沙茶', '保安'},
 '石頭鄉燜烤珍珠玉米(崇德店)': {'乾兒子', '玉米', '石頭', '其他小吃', '沙茶', '保安'},
 '碗稞蘭': {'大俠', '碗粿', '食物美味', '其他小吃', '豬腳', '阿蘭', '麻豆'},
 '碧富邑午晚餐 Beaut

 '金滿 JIN MAN': {'著棋', '服飾店', '藝廊', '金滿', '懶人', '複合式'},
 '金田食堂': {'非常好吃', '壽司', '鮭魚', '食物美味', '丼', '比目魚', '炙燒', '綜合日式料理'},
 '金礦咖啡(台南國賓門市)': {'複合式', '國賓', '捲', '抹茶', '多一分', '金礦'},
 '金礦咖啡(永華店)': {'複合式', '蒼蠅', '永華', '蛋糕', '乳酪', '金礦'},
 '金福氣南洋食堂-台南東寧店': {'吐司', '福氣', '其他異國料理', '南洋', '東寧', '咖椰烤'},
 '金科威補天鮮滴雞湯': {'', '其他美食', '無評'},
 '金站麻辣鍋': {'其他正面情緒',
           '口味清淡',
           '天使',
           '套餐',
           '服務好',
           '梅花',
           '環境好',
           '紅蝦',
           '非常好吃',
           '食物美味',
           '麻辣',
           '麻辣火鍋'},
 '金色三麥': {'價格較高昂',
          '其他正面情緒',
          '南紡夢',
          '啤酒',
          '時代',
          '櫃位',
          '現釀',
          '環境好',
          '酒吧 Lounge Bar',
          '食物美味'},
 '金色印度咖哩屋': {'小份量', '新市區', '新市', '砂鍋', '食物美味', '印度料理', '咖哩', '勛'},
 '金色小屋': {'偏甜',
          '其他正面情緒',
          '旺旺',
          '格芙餅',
          '比利時',
          '西點烘焙、麵包',
          '食物很香',
          '食物精緻漂亮',
          '食物美味',
          '鬆餅',
          '鮮奶油'},
 '金芳布丁':

In [323]:
a=table_df.tags
tagdien=[]
for i in a:
    tagdien+=i
tagdien={tag:set() for tag in set(alltag)}
for dien in dientag:
    for tag in dientag[dien]:
        if tag!="":
            tagdien[tag].add(dien)
#kv >tag:店set
pprint.pprint(tagdien)

{'一下場': {'雅+達藤原豆腐冰'},
 '一中': {'一中街 大輪 紅豆餅'},
 '一串': {'串燒心小炒', '泰式幽靈串燒', '一串心燒烤屋'},
 '一九九八年': {'美樂蛋糕(美樂蛋糕)'},
 '一事': {'FAB義式料理'},
 '一二五': {'二師兄古早滷味'},
 '一二層': {'松笠屋'},
 '一佰年': {'龍泉食堂'},
 '一保堂': {'大菜市純薏仁'},
 '一個紙': {'自來香餡餅(新營店)'},
 '一個肚子': {'村伯菜粽'},
 '一元': {'繆思義式餐坊(原繆思園)', '仁弘宴席'},
 '一到': {'月芽麵包'},
 '一則以喜': {'三皇三家(台南新光三越門市)'},
 '一則以憂': {'三皇三家(台南新光三越門市)'},
 '一剎那': {'安迪的店'},
 '一包': {'林永泰興蜜餞行', '倍樂烘焙坊', '東東鮮蝦餅', '啾甘心濃郁養生茶飲'},
 '一匹': {'一代茶師'},
 '一吃成': {'麥味登 Cafe&Brunch(正南店)'},
 '一味': {'一味品碗粿'},
 '一啟': {'歡樂牛排'},
 '一大家子': {'異人館咖啡部屋(中華店)'},
 '一大截': {'有夠香火雞肉飯'},
 '一大養生麵食': {'味聖豆花'},
 '一夫': {'淳鳩一夫拉麵'},
 '一夾取': {'慈濟園素食'},
 '一對一': {'千尋ちひろ日式餐飲'},
 '一店': {'瑞穗沙茶爐(一店)'},
 '一心': {'林家素食', '一心二葉創意料理部館'},
 '一怒之下': {'麥當勞(台南中華店)'},
 '一恒': {'海之味海產'},
 '一成': {'Double Cheese 披薩吃到飽餐廳'},
 '一手交錢': {'咖哩の蛋蛋'},
 '一手私藏世界紅茶': {'一手私藏世界紅茶(台南東寧店)'},
 '一批': {'日式壽司 惠比壽'},
 '一拿結': {'早鳥手作三明治'},
 '一斤': {'韓氏古早味油飯'},
 '一方': {'一方日朝食X喫茶'},
 '一景': {'田田茶舖(台南店)'},
 '一朗': {'ICHIRO，ㄧ起囉！'},
 '一朝一夕': {'大埔平價鐵板燒(立人店)'},
 '一本': {'一本花田(南門店)'

 '京正拉麵': {'Mr. 拉麵'},
 '京肴': {'京肴燒肉飯'},
 '京都': {'屋衛茶室', '庵六花 京都濃咖哩專賣', '胡攪蝦餃'},
 '京醬': {'食府苑'},
 '京鶴': {'京鶴日本料理'},
 '亭': {'築蠻亭 日式燒肉居酒屋', '鰻の亭 台南現烤鰻魚料理(崇善店)', '海豐亭(海豐亭（台南 府前店）)'},
 '亮綠': {'吉購吉日式丼飯茶飲專賣店-台南府前店'},
 '亮緣色': {'樂饕饕早午餐'},
 '人人有責': {'阿嬤臭豆腐'},
 '人們': {'桃太郎日本料理'},
 '人可食': {'海の味餐廳'},
 '人合菜': {'中華一番'},
 '人圓': {'賞鮮炭火燒美食舖'},
 '人式': {'河童日式居酒屋'},
 '人形': {'新天地'},
 '人情味': {'人情味關廟肉粿'},
 '人會覺': {'Goodday Café'},
 '人美估': {'楊北平脆皮烤鴨烤雞專賣店'},
 '人耶': {'豬8色 韓式料理'},
 '人脈': {'木西米糕店'},
 '人茶': {'開基武廟 Te en Té 山露露茶吧'},
 '人蔘': {'炭吉168韓國料理', '龍鳳東大門', '廟口海產粥'},
 '人話': {'Togo cafe土狗咖啡'},
 '人造奶油': {'Mon Ga Bagel'},
 '人選': {'鍋字號 湖南辣椒鍋'},
 '人雅痞': {'查理TEA SHOP'},
 '人顧店': {'卡堤雅咖啡'},
 '人龍': {'戴張餅吧'},
 '亼': {'黑亼 鑄鐵養生鍋'},
 '什錦': {'上海味香小吃店', '餓勢力廣東粥', '阿波魯麵', '響亮冰舖', '非可披薩', '斯多里中西美食', '阿滿小炒', '維宣素食'},
 '仁之藏': {'開山小火鍋'},
 '仁和': {'饗幸福', '饗の幸福', '喜得(仁和店)', '愛咖客咖啡館', '台灣名點薄皮手工湯包(仁和店)', '紅苑茗 創意日式和洋料理'},
 '仁壽': {'仁壽宮傅家の蚵嗲'},
 '仁弘': {'仁弘宴席'},
 '仁湯': {'莊子土豆仁湯'},
 '今夜': {'下港肉燥店'},
 '今天下午': {'南方牧場'},
 '今牛作': {'今牛作溫體健康爐'},
 '

            '寶日式涮涮鍋(台南安平店)',
            '將太食品',
            '小樽創意日式和洋料理',
            '小知定食屋',
            '屋麵うめえ',
            '山記魚仔店',
            '岐夫部屋',
            '島旬',
            '府城屋台拉麵',
            '捕鯨船',
            '斑鳩的窩',
            '旭屋牛丼專賣',
            '昆布食堂',
            '昌津柴魚',
            '明森宇治抹茶專賣店',
            '星丼食堂(海安店)',
            '暖暖 麻辣 手作關東煮 碳烤',
            '本家台灣咖喱(台南站前店)',
            '東京益田',
            '東洋日式定食',
            '東門町食堂',
            '柊御食事處',
            '桂田酒店-桂田本家日式庭園餐廳',
            '桃山餐廳(健康店)',
            '森日式和食',
            '樂樂屋可樂餅專賣店',
            '樂田樂 日式關東煮',
            '樂禾田(海安店)',
            '樸九鼎風味火鍋',
            '橋北屋日本家庭料理(永福店)',
            '橋野屋はしのや 日式拉麵丼飯',
            '橫須賀海軍咖哩',
            '歐野基屋台',
            '正義餃子',
            '毛丼 丼飯專門店',
            '泉屋日本料理',
            '泰禾平價日本拉麵(西門店)',
            '洋蔥咖哩工房',
            '海洋の章魚燒',
            '海洋的章魚燒',
            '湯咖哩',
            '滿福亭 まんぷくてい',
            '漁人食堂',
    

 '卓家': {'夏家魚麵', '卓家汕頭魚麵意麵(中華總店)', '卓家汕頭魚麵(民生老店)'},
 '協會': {'崁頭山咖啡館'},
 '南京': {'南京牛肉麵館', '大林牛肉麵', '頡寶牛肉麵', '龍鳳牛肉麵'},
 '南傳': {'洪記蒸餃(善化店)'},
 '南勢': {'174翼騎士驛站'},
 '南化區': {'南芝園芒果'},
 '南區': {'巧味鹹酥雞'},
 '南台': {'野太郎日式丼飯'},
 '南商': {'子曰咖啡'},
 '南園': {'和記鍋貼', '鍋貼', '南園街鍋貼'},
 '南大': {'光南大批發(國寶園店)', 'OhDaDa 菓子工坊'},
 '南寧': {'i.mita 愛米塔', '白色曙光 Light Cafe', '桃花源牛肉麵館'},
 '南寶': {'麻辣食堂健康概念館'},
 '南山': {'武英街無名乾麵'},
 '南島夢遊': {'cama 現烘咖啡專門店(台南西門店)'},
 '南投': {'柱老爹'},
 '南方': {'繼光香香雞(台南南方公園)', '小上海香酥雞(南方公園店)', '南方脆皮紅豆餅', '蔥樂手工洋蔥圈', '食內嗑美國南方肯瓊料理'},
 '南棗': {'五條港府城伴手禮'},
 '南極蝦': {'舞鶴燒烤舖'},
 '南泉': {'南泉冰菓室'},
 '南法': {"L'OCCITANE Café 歐舒丹咖啡(南紡夢時代)"},
 '南洋': {'阿香越南美食', '金福氣南洋食堂-台南東寧店'},
 '南洋小館': {'正宗南洋風味小館'},
 '南洋杉': {'山林事務所'},
 '南瀛伴': {'六甲來來黃金酥店'},
 '南瓜': {'131風味廚房',
        '88花園咖啡無菜單創意料理',
        'DACOZ! 大口吃',
        'Double 9 早午晚餐',
        'J&M 咖啡館',
        'MÖVENPICK café 莫凡彼咖啡館(台南南紡夢時代)',
        '京嘆號',
        '半個鍋個人火烤兩吃鍋物(台南海安店)',
        '南瓜餐桌',
        '多加美食',
        '巴布朵食尚館',
        '巴特里精緻烘焙(台南中華店)',

 '大林': {'喵咖啡', '大豆豆麵店', '阿波魯麵', '大林牛肉麵', '大山牛排'},
 '大枝': {'湯家蝦捲', '鄭家老漬號蜜餞'},
 '大校': {'大醬川麵館(成大店)'},
 '大桔': {'和喫GRAZE手燒鬆餅'},
 '大樂': {'Double Cheese 披薩吃到飽餐廳'},
 '大樓': {'魚羊鮮豆'},
 '大橋': {"ST.1 Cafe'", '水鳥客廳', '阿興鹽水雞(大橋夜市)'},
 '大武': {'羊角村手工冰淇淋優格專賣店', '私藏部曲'},
 '大水缸': {'大水缸傳統飲品專賣店'},
 '大河': {'汕頭古早味豆花'},
 '大波浪': {'飛行程是航空主題餐廳'},
 '大洋': {'茶桶餐廳', '永泰興蜜餞行(安平路分店)', '憲壽司', '新都巷日本料理'},
 '大港': {'味傳統豆花'},
 '大潤發': {'丹堤咖啡研磨專賣店 Dante Coffee(大潤發台南店)', '民德虱目魚粥'},
 '大潭': {'牛婆美食部屋'},
 '大灣': {'NU PASTA(台南東寧店)',
        '一派胡塩酵素臭豆腐(大灣店)',
        '台灣茶渠',
        '大灣花生糖(西門店)',
        '大灣隆冰點',
        '尚介讚香雞排(大灣店)',
        '水煎包',
        '進福花生糖',
        '阿麗阿珍大灣名產花生糖'},
 '大火': {'小唐簡餐', '安平海鮮炒麵'},
 '大爺': {'迎大爺扁食'},
 '大牌': {'爆漿打狗棒'},
 '大王': {'新化阿發排骨大王(善化分店)', '（台南店）台東春牛豆漿大王'},
 '大理石': {'老茶園廚坊'},
 '大甲': {'迷客夏 Milk shop', '炒翻天(育樂店)'},
 '大發': {'金大發韓式炸雞脆皮年糕'},
 '大盒': {'臻品香現烤蛋糕'},
 '大盛': {'屋麵うめえ'},
 '大目': {'真心食府'},
 '大眼': {'日日咖啡', 'why東西'},
 '大眾豬': {'陶湘涮涮鍋'},
 '大眾鍋': {'牛墟火鍋、茶飲、咖啡'},
 '大福': {'85度C咖啡蛋糕烘焙專賣店(台南北門店)',
  

...

         '田林日本料裡',
         '紙的部屋咖哩輕食',
         '蛋包飯專賣屋',
         '遇見幸福異國創意料理',
         '遇見果香芒果冰',
         '食べオム。ta be o mu 洋食歐姆蓋飯專賣店',
         '食べオム洋食歐姆蓋飯専売',
         '高雄空廚(台南大遠百成功店)'},
 '蛋圈': {'石堂極和牛石頭燒'},
 '蛋堡': {'炸鷄洋行(榮譽店)'},
 '蛋塔': {'PS oven',
        '肯德基 KFC(台南中華)',
        '肯德基 KFC(台南西門店)',
        '肯德基/台南東門餐廳',
        '肯德基/台南永康餐廳',
        '虎山雷創意蛋塔'},
 '蛋塔要': {'城門爆漿雞蛋燒'},
 '蛋夯': {'吐司找茶'},
 '蛋夾': {'洪瑞珍(台南店)'},
 '蛋屋': {'話不多咖啡小小小空間'},
 '蛋殼': {'姑姑茶葉蛋(原芝的茶葉蛋)'},
 '蛋汁秀': {'58鍋燒'},
 '蛋液': {'沈味香古早味'},
 '蛋湯': {'集品蝦仁飯', '矮仔成蝦仁飯', '集鮮蝦仁飯'},
 '蛋炒飯': {'麒珍素食(善化店)', '南方脆皮紅豆餅', '上海華都小吃點心城', '飯賣人口', '炒飯專家', '安平海鮮炒麵'},
 '蛋烤': {'松仔腳燒烤'},
 '蛋皮': {'京樽壽司', '蛋包飯專賣屋'},
 '蛋皮裡': {'蛋包飯專賣屋'},
 '蛋糕': {'85度C咖啡蛋糕烘焙專賣店(台南北門店)',
        '85度C咖啡蛋糕烘焙專賣店(台南新市店)',
        '85度C咖啡蛋糕烘焙專賣店(永康中華店)',
        '85度C咖啡蛋糕烘焙專賣店(永康南科店)',
        '99 Novantanove',
        'ChuChu Pâtisserie 法式甜點',
        'Dessert around US',
        'Do - Li  BAKERY CAFE 多麗咖啡烘焙美食',
        'Dreaming cafe 綺飛 法式甜點',
        'HO-YO Cook

 '辣肉': {'老王牛肉麵'},
 '辣肉味': {'一風堂拉麵(台南西門店)'},
 '辣腸': {'麻豆驢子廚房民宿'},
 '辣菜': {'小師傅池上米食'},
 '辣豆': {'Kook美式餐廳', '晚起餐館'},
 '辣豬': {'燒出名堂'},
 '辣辣脆': {'小南雞腿飯(鹽行店)'},
 '辣醬': {'麥當勞(善化店)', '東春館韓式料理', '南邦泰式平價料理', '치킨샾 Chicken shop 韓國炸雞專賣'},
 '辣雞': {'允記食堂'},
 '辦公室': {'CHOW CHOW BAGEL'},
 '辦公椅': {'拾參'},
 '辦桌': {'梅鳳飲食店', '各種筵席包辦', '欣福龍海鮮餐廳', '府城食府'},
 '辰星': {'辰星咖啡'},
 '農場': {'一芝紅大旺來鳳梨酥烘焙坊', 'Goodday Café'},
 '農家': {'樹谷懷舊餐廳(樹谷園區)'},
 '農庄': {'南勢農庄龍眼乾'},
 '農店': {'小時候綠豆湯'},
 '農會': {'熱情小子芒果冰館', '新市農會超市'},
 '農村': {'翁稼茶棧'},
 '農業': {'翁稼茶棧'},
 '農舍': {'新化農舍餐廳'},
 '農舍庭園餐廳': {'新化農舍餐廳'},
 '農藥': {'三角窗'},
 '農青': {'梅嶺食品中心'},
 '迅速': {'定食8 爭鮮餐飲(台南店)'},
 '迌': {'暖暖蛇咖啡館 Café Flâneur(原迌咖啡)'},
 '迎大爺扁食': {'奇香蚌麵'},
 '迎春': {'府城食府'},
 '迎曦': {'水岸人文休閒咖啡館'},
 '迎記': {'迎記麻辣鴛鴦火鍋(永康分店)'},
 '迎面而來': {'新港無骨鴨肉羹'},
 '近乎': {'遇見幸福異國創意料理'},
 '近博': {'艸疏田木'},
 '近畿': {'異人館咖啡部屋(成功店)'},
 '近親': {'滇緬料理'},
 '迦南': {'迦南水果店'},
 '迪克': {'108巷咖啡',
        '137 Bookstore 咖啡館',
        'Deli Farm 天田輕食館',
        'FILM 茀立姆早午餐',
        'Good day 輕食咖啡下午茶',
     

 '雞酒': {'上品藥膳美食'},
 '雞鍋': {'弋霸不冷鍋物專賣店', '龍鳳東大門'},
 '雞飯': {'Cafe Mood',
        'INS飲食坊',
        '南洋小館',
        '嘉洲食坊-新加坡美食',
        '正宗南洋風味小館',
        '長榮快餐',
        '馬來峰巴生肉骨茶'},
 '雞骨': {'屋麵うめえ'},
 '雞魂': {'雞魂無骨油雞、醉雞(台南公園店)'},
 '雞魯飯': {'山種子'},
 '難吃': {'木匠炭火燒肉'},
 '難喝': {'綠林'},
 '難放久': {'古留香狀元粿'},
 '難調': {'壽喜屋'},
 '難養': {'阿鳳碗粿'},
 '雨來': {'稻稻來社區廚房'},
 '雨林': {'Double Yummy'},
 '雨神': {'Angry Burger  美式餐廳',
        'Dacci 打趣法式廚房',
        'YES MARIE Bistro 也是瑪莉',
        '佳里御品紅豆',
        '信燒烤',
        '吉米thai 泰式料理',
        '咖哩太郎 日式咖哩專賣(國賓店)',
        '咖啡平方隨行吧(崇德店)',
        '天下大飯店',
        '山本堂日式拉麵',
        '御私藏鮮奶茶專賣店(安平店)',
        '津華香香鍋-永大店',
        '知點新時尚料理餐廳',
        '碳味亭和牛炭火燒肉',
        '福村家庭小料理',
        '福豆屋(德安店)',
        '老夫子牛肉麵 成功店',
        '蘭姐1899關東煮串燒',
        '虹橋餐廳',
        '阿蜜家廚料理',
        '陶林日式涮涮鍋',
        '雙角號',
        '響道食堂',
        '麻古茶坊',
        '黑皮太'},
 '雪乳冰': {'維美雪乳冰'},
 '雪倫': {'牛狀元牛肉湯'},
 '雪印': {'關西川本家居酒屋 x 大阪燒'},
 '雪媽': {'黑人洪鵝肉'},
 '雪山': {'紅豆屋'},
 

          '聖加南洋小品(台南新光三越西門店)',
          '聖記軟骨飯專賣店',
          '聚樂居食屋',
          '聯盈發點心專門店',
          '肉伯火雞肉飯',
          '肉皮張滷味(台南忠義店)',
          '肉皮張魯味(金華店)',
          '肕 燒肉',
          '胡攪蝦餃',
          '自己的房間',
          '臺南三股四季海產店',
          '臺南大飯店 - 府城食選任意點',
          '臺灣鮮魚湯',
          '臺邦商旅-帕莎西餐廳',
          '臻品香現烤蛋糕',
          '臻皇港式餐廳',
          '舒果新米蘭蔬食(台南府前店)',
          '舒芙里法式烘培坊',
          '舞飛日式燒肉',
          '舟鶴家嫩骨飯(文賢店)',
          '艋舺雞排(台南中正店)',
          '良物肉鋪子',
          '艾摩多手工杏仁豆腐專賣店',
          '艾維瓦義式廚房',
          '芋豆賞',
          '芙聿閣菓子屋',
          '芭樂芭 成大店(成大店)',
          '芭農焗烤屋',
          '花川日本料理',
          '花見宇治抹茶甘味処',
          '花見小路',
          '芳苑冰棒',
          '芳鄰小館',
          '茂記安平黑豆花',
          '茂雄蝦仁肉圓',
          '莉莉水果店',
          '莊子土豆仁湯',
          '莎麗泰式家庭餐廳',
          '菓爾Pasta',
          '華侖婷娜',
          '華味香(三店)',
          '萬客樓餐館',
          '萬川號',
          '萬福樓餐館',
          '萬紅刀切麵',
          '萬香土產紅燒羊肉爐',
          '落成米糕店',
  

 '點至': {'大頭簡餐', '山泉鹽水雞', '太古●TAIKOO'},
 '點薄皮': {'50 PIZZA(新營中正店)'},
 '點裡面': {'阿滿素食'},
 '點豚': {'貳木'},
 '點醋': {'府城浮水虱目魚焿'},
 '點鍋': {'鍋寶臭臭鍋(武聖店)'},
 '點鍋燒': {'早安！美芝城(新市中正分店)'},
 '點餐機': {'定食8 爭鮮餐飲(台南永康店)'},
 '黯然消魂': {'功夫啦哩', '香港角港式茶餐廳(東豐店)'},
 '黯然銷魂': {'功夫啦哩'},
 '鼎': {'何家鼎邊趖'},
 '鼎味': {'鼎味創意食尚'},
 '鼎富': {'鼎富臭豆腐'},
 '鼎極': {'鼎極鴛鴦鍋鐵板燒(新營店)', '鼎極極地海鮮日本料理'},
 '鼎泰豐': {'泰豐燒臘便當'},
 '鼎記': {'珍珠工場(南工店)'},
 '鼎鼎大名': {'鼎鼎大名火鍋專賣(永大店)'},
 '鼓豆': {'鼓豆咖啡 Great Bean Company'},
 '鼻塞': {'風庭園砂鍋粥'},
 '鼻子': {'豬鼻子越式創意美食(無)', '奶油鼻子輕食烘焙坊'},
 '齁': {'鮮魚蔥肉粿'},
 '齊女': {'明仁鍋貼'},
 '齊齊': {'85度C咖啡蛋糕烘焙專賣店'},
 '齋': {'食蔬茶齋 蔬果料理'},
 '龍之味': {'龍之味炒飯燴飯'},
 '龍井': {'上海居食之家'},
 '龍山': {'ㄌㄜˋ腳王', 'mini coffee', '築地壽司', 'is義式藝術餐廳', 'Y平方輕食', '七股阿宗蚵嗲蚵仔麵線'},
 '龍崎': {'青龍山土雞城', '中一豆花', '優里司複合式餐飲'},
 '龍年': {'日和藏'},
 '龍潭': {'邱媽媽花生糖'},
 '龍珠': {'夜貓子啤酒屋'},
 '龍田': {'小碗和食'},
 '龍田川': {'一汁三菜'},
 '龍田揚': {'一汁三菜'},
 '龍眼': {'南勢農庄龍眼乾'},
 '龍眼乾': {'南勢農庄龍眼乾'},
 '龍眼肉': {'南勢農庄龍眼乾'},
 '龍膽': {'料理是實 Is Truth', '三和鮮魚湯', '蒸鍋蒸煮健康鍋', '粥皇京樓', '水鳥客廳', '文南森深海鮮魚湯'},
 '龍興': 

In [324]:
len(tagdien['食物美味'])

1588

In [325]:
#r.ltrim('tutorial-list',0,-1000000)刪除list的方法(最後100000筆)
for dien in dientag:
    r.ltrim(dien,0,-1000000)
for tag in tagdien:
    r.ltrim(tag,0,-1000000)
#先刪除以免重覆加入
    
    
#將dientag跟tagdien分別加入redislist
for dien in dientag:
    r.lpush(dien,*dientag[dien])
for tag in tagdien:
    r.lpush(tag,*tagdien[tag])

In [359]:
#1用屬性找店家
querytags=input("請輸入想要的屬性(可輸入多個，以空白分開):")
if querytags!="":
    dienList=[]
    
    #將輸入的string變成taglist
    for query in set(querytags.split(" ")):
        print(query)
        #將情緒同義字對應query
        if query in pinsynonymsdict:
            query=pinsynonymsdict[query]
            
        #一個query可以找到一個店家list
        if len([i.decode() for i in r.lrange(query,0,10000)])==0:
            for tag in alltag:
                if query in tag:
                    query=tag
                    break
        print("實際tag："+query)
        if dienList==[]:
            dienList =[i.decode() for i in r.lrange(query,0,10000)]
        else:
        #多個query可以找到不同店家list得交集
            dienList = set(dienList) & set([i.decode() for i in r.lrange(query,0,10000)])
#     print('屬性搜索項目:')
#     print(dienList)

#2用店名找店家
querydeins=input("查店名:")
if querydeins!="":
    nameList=[]
    for dein in set(querydeins.split(" ")):
        print(dein)
        if nameList==[]:
            nameList =[j for j in [i.decode() for i in r.keys()]  if dein in j]
        else:
            nameList = set(nameList) & set([j for j in [i.decode() for i in r.keys()]  if dein in j])
#     print('店名搜索項目:')
#     print(nameList)
    
if querytags!="" and querydeins!="":
    allquery=set(dienList) & set(nameList)
elif querytags!="" and querydeins=="":
    allquery=set(dienList)
elif querytags=="" and querydeins!="":
    allquery=set(nameList)
    
print('綜合搜索項目:')
#     print(allquery)
queryscore=Counter({dien:dienscore[dien] for dien in dienscore if dien in allquery}).most_common(10)
pprint.pprint(queryscore)


請輸入想要的屬性(可輸入多個，以空白分開):其他小吃 美味 便宜
便宜
實際tag：划算
美味
實際tag：食物美味
其他小吃
實際tag：其他小吃
查店名:
綜合搜索項目:
[('日光 緩緩', 31.67), ('舟鶴家嫩骨飯(文賢店)', 27.97), ('豆哥牛排館', 26.97), ('一點刈包', 9.38)]


In [360]:

for i in bigTable:
    if i['name'] in dict(queryscore).keys():
        print(i['name'])
        print(i['style'])
        print(i['score'])
        print(i['introduction'])
        print(i['featurescores'])
#         print(i['tags'])
        print('-'*50)
#         print(i['name'])

舟鶴家嫩骨飯(文賢店)
其他小吃
27.968421052631555
【舟鶴家嫩骨飯（文賢店）】，2012年創立中華店，2015年開設文賢分店，始終堅持台灣傳統好滋味，店名取自台語諧音「真好吃」，主打以多種中藥配方調理熬製的嫩骨飯、片鴨飯，親民平價的實惠消費，吸引廣大顧客歡迎
{'食物美味': 1.07, '划算': 0.57, '環境好': 0.98, '食物很香': 0.46, '其他正面情緒': 0.59, '份量大': 0.68, '小份量': 0.24, '偏辣': 0.08, '食物精緻漂亮': 0.16, '重口味': 0.5, '服務好': 0.2}
--------------------------------------------------
一點刈包
其他小吃
9.377777777777782
一點刈包是一個有宵夜、平價、吃吃喝喝的其他小吃，網友認為值得推薦的有：豆漿、刈包、蛋餅、刈包加蛋、蘿蔔糕
{'食物美味': 0.5, '其他正面情緒': 0.16, '偏甜': 0.19, '划算': 0.45, '食物精緻漂亮': 0.13, '食物很香': 0.2, '環境好': 0.11, '份量大': 0.19, '小份量': 0.41, '重口味': 0.08}
--------------------------------------------------
日光 緩緩
其他小吃
31.673684210526286
日光緩緩是一個有早午餐、brunch、免服務費的其他小吃，網友認為值得推薦的有：超值早餐、天使細麵、健康蜂蜜醋、沙拉、今日湯
{'食物美味': 1.13, '划算': 0.95, '環境好': 0.82, '食物精緻漂亮': 1.0, '其他正面情緒': 0.93, '份量大': 0.8, '偏甜': 0.33, '小份量': 0.54, '服務好': 0.11, '食物很香': 0.23, '重口味': 0.74, '偏酸': 0.33, '偏辣': 0.08}
--------------------------------------------------
豆哥牛排館
其他小吃
26.972727272727294
喜歡大口吃肉的朋友有福了！台南超美味又平價的【豆哥牛排館】，希望能以好的肉品讓顧客吃得幸福！這裡

In [327]:
a=['d', 'c', 'b', 'a', 'MySQL', 'Mangodb', 'Redis', 'MySQL', 'Mangodb', 'Redis',"笨蛋"]
b=["笨蛋", 'Redis','yayaya']
list(set(a) & set(b)) #交集

['Redis', '笨蛋']

In [328]:
list(set(a) | set(b)) #聯集

['yayaya', 'Mangodb', 'b', 'a', 'c', 'MySQL', 'd', 'Redis', '笨蛋']

In [329]:
#list包函於list2 判斷的方法
set(['d', 'c', 'b']) & set(a)==set(['d', 'c', 'b'])

True

In [330]:
x='川品鐵板'
for i in bigTable:
    if x in i["name"]:
        print(i['tags'])
        print(i['score'])

['份量大', '海陸', '環境好', '非常好吃', '食物很香', '其他正面情緒', '鐵板', '划算', '川品', '鐵板燒', '套餐', '其它中式料理']
34.533333333333346
